In [1]:
import pandas as pd                               # panda
import pandas_datareader as pdr                   # Baixar dados yahoo

import matplotlib.pyplot as plt                   # grafico
import matplotlib.dates as mdates                 # trabalhar datas

import statsmodels.api as sm                      # regressao
from statsmodels.api import add_constant          # para a regressao considerar o intercepto

import math as math
import numpy as np
import scipy.stats


from statsmodels.tsa.stattools import coint       # Funcao Coint ()
from statsmodels.tsa.stattools import adfuller

# Declaracoes
pd.options.display.float_format = '{:.2f}'.format

#!pip install MetaTrader5
#!pip install --upgrade MetaTrader5
import MetaTrader5 as mt5
from datetime import datetime
from datetime import date
from datetime import timedelta
import time
import pytz
import sys
import timeit
import ipywidgets

CALENDARIO_B3 = pd.read_excel('Calendar.xlsx')


%matplotlib inline

In [2]:
def Get_data(tickers,inicio,fim,janela):

    # Checa se ja estamos conectados, se não, conecta
    if (mt5.terminal_info()==None):
          
        # conecte-se ao MetaTrader 5
        if not mt5.initialize():
            print("initialize() failed, error code =",mt5.last_error())
            mt5.shutdown()
        
    #quebra as strings dates(inicio,fim) recebidas e transformando em integer
    inicio_= inicio.split('-')
    inicio = [int(element) for element in inicio_]
    fim_= fim.split('-')
    fim = [int(element) for element in fim_]

    timezone = pytz.timezone("Etc/UTC")
    
    #Verifica caso seja timeframe diario ou intraday
    if janela == '1d':
        dt_inicio = datetime(inicio[0], inicio[1], inicio[2], tzinfo=timezone)
        dt_fim = datetime(fim[0], fim[1], fim[2], tzinfo=timezone)
        timeframe = mt5.TIMEFRAME_D1
    #aqui trata as datas intraday
    else:
        dt_inicio = datetime(inicio[0], inicio[1], inicio[2], inicio[3], inicio[4], tzinfo=timezone)
        dt_fim = datetime(fim[0], fim[1], fim[2], fim[3], fim[4], tzinfo=timezone)
        timeframe = mt5.TIMEFRAME_M15
        
    # obtendo o par de acoes com base nas datas enviadas nas strings, caso len seja 1, getdata retorna o dataframe de uma unica acao
    if (len (tickers) == 1):        
        stock1 = mt5.copy_rates_range(tickers[0], timeframe, dt_inicio, dt_fim)    
        stock_df = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
                
    else:    
        stock1 = mt5.copy_rates_range(tickers[0], timeframe, dt_inicio, dt_fim)
        stock2 = mt5.copy_rates_range(tickers[1], timeframe, dt_inicio, dt_fim)
        esquerda = pd.DataFrame(stock1)[['close','time']].rename(columns={'close': tickers[0]})
        direita = pd.DataFrame(stock2)[['close','time']].rename(columns={'close': tickers[1]})
        stock_df = pd.merge(left=esquerda, right=direita, left_on='time', right_on='time')
    
    #Se o timeframe for intraday, retorna um datetime (com minutos e horas), caso contrario, retorna um date
    if (janela != '1d'):
        stock_df['Date'] = stock_df['time'].apply(lambda x: str (datetime.fromtimestamp(x, tz=timezone))[:-9]) #define timezone
    else:
        stock_df['Date'] = stock_df['time'].apply(lambda x: date.fromtimestamp(x+21600)) #adicionando 21600 pra corrigir o fuso no momento de converter tsp em str
        stock_df.drop('time',axis=1,inplace = True)
        
    return stock_df.set_index('Date')

In [3]:
# Gera a REGRESSAO LINEAR MULTIVARIADA (QuantGo "Simples") -> y = b*x + c*t + d + e
def regressao_multivariada(x, y, periodo):  
    
    X = np.column_stack((x, range(1,periodo+1,1)))
    X = sm.add_constant(X)
    res = sm.OLS(y,X).fit()
    coef_ang=res.params[1]
    residuo = y-res.predict()
    zscore  = (residuo - np.mean(residuo))/np.std(residuo);
    return coef_ang, residuo, zscore, res


def regressao_residuos(residuos):
  
    residuos_shifted = residuos.diff(1).fillna(method="bfill")  
    delta = residuos - residuos_shifted
    X = sm.add_constant(residuos_shifted, prepend=True)
    res = sm.OLS(residuos,X).fit()
    return res.params[1]

# ADF da Statsmodel e teste de significancia com os critical values do Ferro
def adftest(df,reg):

    adf  = adfuller(df,maxlag=1, autolag="BIC")
    
    adf_stat = adf[0]
    adf_pvalue = adf[1]
    
    ## %ADF clássico, de acordo com o output da função  - NAO UTILIZADO
    if   adf[0] < adf[4]['1%']:  adfc='99%'
    elif adf[0] < adf[4]['5%']:  adfc='95%'
    elif adf[0] < adf[4]['10%']: adfc='90%'
    else:                        adfc='-'                 

    ## %ADF do Ferro
    # Utiliza duas tabelas - uma para quando o tempo é significante, outra quando não é (indep. do noobs)
    ttest = reg.params[1]/reg.bse[1]
    critical_value_tempo = scipy.stats.t.ppf(0.01,df=(len(df)-3)) # Retorna o inverso bicaudal da distribuição t de Student

    if (abs(ttest) < critical_value_tempo): aceitar_t0 = '99%'
    else:  aceitar_t0 = '0%'
    
    if (aceitar_t0 == '0%'):
        if   adf[0] < -4.32:         adf_sign='99%'
        elif adf[0] < -3.67:         adf_sign='95%'
        elif adf[0] < -3.28:         adf_sign='90%'
        else:                        adf_sign='-' 
    else:    
        if   adf[0] < -3.58:         adf_sign='99%'
        elif adf[0] < -3.22:         adf_sign='95%'
        elif adf[0] < -2.60:         adf_sign='90%'
        else:                        adf_sign='-'                 

    
    
        
    return adf_stat, adf_sign, adf_pvalue  


def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''
    
    r, p = scipy.stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = scipy.stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    
    return r, p, lo, hi
    
def pct_financeiro(x,y,coef_ang,residuo):
    
    ultimo_x = float(x.tail(1))
    ultimo_y = float(y.tail(1))
    
    fin_x = ultimo_x*coef_ang
    fin_y = ultimo_y
    
    if (residuo > 0):
        compra = fin_x
        venda = fin_y
    else:
        compra = fin_y
        venda = fin_x
 
    cv = "{:.0%}".format(compra/venda)
    return cv


def calculo_meia_vida1(residuo):
    # Retirado de um paper e adaptado pela formula da planilha do Ferro
    price = pd.Series(residuo)  
    lagged_price = price.shift(1).fillna(method="bfill")  
    delta = price - lagged_price  
    beta = np.polyfit(lagged_price, delta, 1)[0] 
    #half_life = ((-2*np.log(2))/beta)  # paper (varios), mas sem o 2*
    half_life = 2/(-1*np.log(1+beta))   # planilha ferro
     
    return (int(round(half_life)))


In [4]:
def ajusta_B3Date (data_start, qtde_dias):
    
    df = CALENDARIO_B3
    df['Date'] = df['Date'].apply(str)
    try:
        index = df [df['Date'] == data_start].index.values.astype(int)[0]
        data_fim = df.iloc[index+qtde_dias][1]
    except:
        print ('****************************************************************')
        print ('ERRO, TENTATIVA DE ACESSO DE DATAS NAO INCLUIDAS EM CALENDAR.XLS')
        print ('****************************************************************\n')
        sys.exit()
    else:
        return (data_fim + '-17-00')

    
def simula_trade (ticker, beta_inicio, dp_entrada, dp_target, start_time , hlife):
    
    #passando a data de inicio do trade para identificarmos qual a data limite para o trade com base no calendario da B3
    end_time = ajusta_B3Date(start_time[:-6],hlife)
    #print (start_time)
    #print (end_time)
    stock = Get_data(ticker, start_time, end_time,'M5')
    
    return stock
    
def merge_intraday (ticker, dt_inicio, dt_intraday):

    dt_intraday_ant = datetime.strptime(dt_intraday, '%Y-%m-%d-%H-%M') 
    dt_intraday_ant = dt_intraday_ant - timedelta(days=1)  # Converte p/ data e subtrai 1 dia
    dt_fim = dt_intraday_ant.strftime('%Y-%m-%d-%H-%M')
    
    stock  = Get_data(ticker, dt_inicio,dt_fim,'1d')
    stock_intraday = Get_data(ticker, dt_intraday,dt_intraday,'M15')
    
    stock_intraday['Date'] =  stock_intraday['time'].apply(date.fromtimestamp)
    stock_intraday.set_index('Date', inplace = True)
    stock_concat = pd.concat([stock,stock_intraday])
    stock_concat.drop('time',axis=1,inplace = True)

    return stock_concat

In [5]:
#######################################################
#        CONSTRUCAO DE VARIAVEIS TEMPORAIS            #
#######################################################
# - ADFCOUNT  - Qtd Dias com ADF=99% em cada período  #
# - BETAVOL63 - Vol do beta em janela de 63 dias      #
# - BETAVOL10 - Vol do beta em janela de 10 dias      #
# ** TODAS BASEADAS EM FECH DIARIOS ('19-00')         #
#######################################################

def gera_variaveis_temporais(cubototal):

    start = time.time()
    for periodo in [100,120,140,160,180,200,220,240,250]:

        adfcount = 0    
        cubo_tmp = cubototal[cubototal['periodo']==periodo]
        cubo_tmp_19h = cubototal[ (cubototal['periodo']==periodo) & (cubototal['data'].apply(lambda x: x[-5:]) == '19-00') ] # Bet Rot

        date_unique = cubo_tmp['data'].unique()

        for date in date_unique:

            row = cubo_tmp.loc[cubo_tmp['data'] == date]
            horario = row['data'].str[-5:].values[0]
            adf = row['adf_sign'].values[0]

            # ADFCOUNT - Se for 99% quando o horario for 19h, soma 1, senao recomeça a contagem
            if (horario == '19-00'):
                if (adf == '99%'): adfcount += 1
                else: adfcount = 0

            # BETAVOL - concatena a serie de fechamentos (19h) com o ponto atual e calcula a volatilidade percentual
            last_rows_19h = cubo_tmp_19h.loc[cubo_tmp_19h['data'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d-%H-%M')) < datetime.strptime(date, '%Y-%m-%d-%H-%M')]   # Beta Rot

            beta63 = pd.concat([last_rows_19h,row]).tail(63) # Beta Rot   ALTERAR PARA 0.2*PERIODO
            #beta10 = pd.concat([last_rows_19h,row]).tail(10) # Beta Rot   ALTERAR PARA 0.2*PERIODO
            betavol63 = beta63['coef_ang'].std() / beta63['coef_ang'].mean()   # Beta Rot
            #betavol10 = beta10['coef_ang'].std() / beta10['coef_ang'].mean()   # Beta Rot

            # Cria vars de faixas dos betavol10 e betavol63 - até 0.05; entre 0.05 e 0.10; entre 0.10 e 0.20; acima de 0.20
            if   (betavol63 <= 0.05): fx_betavol63 = '<= 0.05'
            elif (betavol63 <= 0.10): fx_betavol63 = '<= 0.10'
            elif (betavol63 <= 0.20): fx_betavol63 = '<= 0.20'
            else: fx_betavol63 = '> 0.20'


            cubototal.loc[ (cubototal['periodo']==periodo) & (cubototal['data'] == date) , 'adfcount'    ] = adfcount
            cubototal.loc[ (cubototal['periodo']==periodo) & (cubototal['data'] == date) , 'betavol63'   ] = betavol63
            cubototal.loc[ (cubototal['periodo']==periodo) & (cubototal['data'] == date) , 'fx_betavol63'] = fx_betavol63
            cubototal.loc[ (cubototal['periodo']==periodo) & (cubototal['data'] == date) , 'betavol10'   ] = betavol10
            cubototal.loc[ (cubototal['periodo']==periodo) & (cubototal['data'] == date) , 'fx_betavol10'] = fx_betavol10
          #
    end = time.time()
    print('Tempo total = {total}'.format(total = (end - start)))

    return cubototal


In [6]:
def Get_Cubo (tickers, dt_cubo, intra_bt):
    
    #Pegando a data B3 de 250 regoes atras
    if intra_bt:
        dt_inicio = ajusta_B3Date(dt_cubo[:-6],-250)
        datahora = dt_cubo
        
        #Para retornar o par com dados diarios, com ultimo dia sendo o intraday passado em dt_final
        par = merge_intraday (tickers, dt_inicio, dt_cubo)
        
    else:
        dt_inicio = ajusta_B3Date(dt_cubo,-250)
        datahora = dt_cubo+'-19-00'
        
        # Busca cotacoes 
        par = Get_data(tickers, dt_inicio, dt_cubo,'1d')
        
    periodos_coint = periodos_coint_99 = 0
    cubo_dict = {}
    beta = []
    betacoint = []
    tempo=0
    # Popula o cubo de periodos
    #for i in range(100,260,10):
    for i in [100,120,140,160,180,200,220,240,250]:

        # Cria subset do tamanho do periodo atual da iteração e seta a variavel indep (x) e dependente (y)
        par_subset = par.tail(i)
        x = par_subset[tickers[1]]
        y = par_subset[tickers[0]]

        # Gera a REGRESSAO e retorna o Coef Ang, os residuos e o z-score. 
        # Retorna tbm o vetor de retorno da regressao para ser usado no ADF
        coef_ang, residuo, zscore, reg  = regressao_multivariada(x, y, i)

        # Teste de Estacionariedade dos Resíduos (ADF)
        adf_stat,adf_sign,adf_pvalue = adftest(residuo, reg)
         
        # Calculo do Fisher
        fisher_r, fisher_pvalue, fisher_lo, fisher_hi = pearsonr_ci(x.diff().fillna(method="bfill"), y.diff().fillna(method="bfill"))

        # Calculo do % Financeiro (C/V)
        pct_fin = pct_financeiro(x,y,coef_ang, float(zscore.tail(1)))

        # Calculo da Meia-vida - ORNSTEIN-UHLENBECK
        meia_vida = calculo_meia_vida1(residuo)
        
        # Correlação 
        #corr = np.corrcoef(x.pct_change().cumsum().dropna(), y.pct_change().cumsum().dropna())[1,0]
        
        # Se fisher minimo menor que 10%, ou se correlacao < 0, faz adf_sign=0%
        if ((fisher_lo <= 0.1) or (coef_ang < 0)): adf_sign = "-"  #or (corr<=0) 
       
        cubo_dict[datahora+'-'+str(i)] = {'data'                   : datahora,
                                            'periodo'              : int(i),
                                            'preco_y'              : float(y.tail(1)),
                                            'preco_x'              : float(x.tail(1)),
                                            'ratio'                : float(y.tail(1))/float(x.tail(1)),
                                            'coef_ang'             : coef_ang,
                                            'desvio'               : float(zscore.tail(1)),
                                            'adf_stat'             : adf_stat,
                                            'adf_sign'             : adf_sign,
                                            'pct_fin'              : pct_fin,
                                            'meia_vida'            : meia_vida,
                                            'dp_residuo'           : np.std(residuo),
                                            'per_coint'            : None,
                                            'per_coint99'          : None,
                                            'vol_beta_vertical'    : None,
                                            'fx_vol_beta_vertical' : None} 

        if (adf_sign != "-"  ): periodos_coint    += 1
        if (adf_sign != "99%"): periodos_coint_99 += 1
            
        beta.append(coef_ang)
        if (adf_sign != "-"): betacoint.append(coef_ang) 

    
    ######### Criando algumas variaveis globais do cubo atual para o backtest
    for i in [100,120,140,160,180,200,220,240,250]: 
        
        # Periodos cointegrados e Periodos com ADF = 99%
        cubo_dict[datahora+'-'+str(i)]['per_coint'] = periodos_coint   
        cubo_dict[datahora+'-'+str(i)]['per_coint99'] = periodos_coint_99   
        
        vol_beta_vertical = np.std(beta)/np.mean(beta)
         
        if   (vol_beta_vertical <= 0.05): fx_vol_beta_vertical = '<= 0.05'
        elif (vol_beta_vertical <= 0.10): fx_vol_beta_vertical = '<= 0.10'
        elif (vol_beta_vertical <= 0.20): fx_vol_beta_vertical = '<= 0.20'
        else: fx_vol_beta_vertical = '> 0.20'      
            
        cubo_dict[datahora+'-'+str(i)]['vol_beta_vertical']  = vol_beta_vertical
        cubo_dict[datahora+'-'+str(i)]['fx_vol_beta_vertical']  = vol_beta_vertical

#         # Vol do Beta dos periodos cointegrados
#         if (len(betacoint)>0): 
#             vol_beta_vertical1  = np.std(betacoint)/np.mean(betacoint)
        
#             if   (vol_beta_vertical1 <= 0.05): fx_vol_beta_vertical1 = '<= 0.05'
#             elif (vol_beta_vertical1 <= 0.10): fx_vol_beta_vertical1 = '<= 0.10'
#             elif (vol_beta_vertical1 <= 0.20): fx_vol_beta_vertical1 = '<= 0.20'
#             else: fx_vol_beta_vertical1 = '> 0.20'  
                
#             cubo_dict[datahora+'-'+str(i)]['vol_beta_vertical1'] = vol_beta_vertical1
#             cubo_dict[datahora+'-'+str(i)]['fx_vol_beta_vertical1'] = vol_beta_vertical1
    
    #display(cubo_dict)
    #display(pd.DataFrame.from_dict(cubo_dict, 'index'))
    return cubo_dict

In [7]:
def geraCubodecubos(tickers, dtinicio, dtfim):

    start = time.time()

    cubo_dict = {}
    
    #### CUBO FECHAMENTOS ####
    datas = Get_data(tickers, dtinicio, dtfim,'1d').reset_index()['Date']

    for dat in datas:
        cubo_dict.update(Get_Cubo(tickers,str(dat),False))
        
    
    #### CUBO INTRADAY ####
    datahora = Get_data(tickers,str(dtinicio+'-10-00') , str(dtfim+'-17-00'),'15m').reset_index()['Date']

    for dthora in datahora:

        # Convertendo 2018-01-02 10:30 para 2018-01-02-10-30
        dthora1 = dthora.split(' ')
        dthora2 = dthora1[0]+"-"+dthora1[1]
        dthora1 = dthora2.split(':')
        dthora2 = dthora1[0]+"-"+dthora1[1]
        cubo_dict.update(Get_Cubo(tickers,str(dthora2),True))
       

    # Transformando em DataFrame() e ordenando
    cubo_total = pd.DataFrame.from_dict(cubo_dict, 'index')
    cubo_total.reset_index(drop=True, inplace=True)
    cubo_total = cubo_total.sort_values(['data','periodo'])

    #display(cubo_total)

    end = time.time()
    print('Tempo total em minutos = {total}'.format(total = (end - start)/60))
    
    ################################################
    # Gera variaveis temporais (DEMORA..otimizar)  #
    ################################################
    
    #cubo_total = gera_variaveis_temporais(cubo_total)
    
    
    return cubo_total

In [8]:
def Busca_trade(cubo_of, coint=2, dp=2., adf_sign =0.95, meia_vida = 10, voltando = False):
    
    cubo_of['date_obj'] = cubo_of['data'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d-%H-%M')) 
    date_unique = cubo_of['date_obj'].unique()
    
    trade_select    = pd.DataFrame(columns=['data','preco_y','preco_x','ratio','periodo', 'adf_stat','adf_sign','coef_ang',
                                             'desvio','pct_fin','meia_vida', 'corr', 'fx_corr','per_coint','vol_beta_vertical','fx_vol_beta_vertical',
                                             'vol_beta_vertical1','fx_vol_beta_vertical1','adf99_periodos'])

    full_trade_select = pd.DataFrame(columns=['data','preco_y','preco_x','ratio','periodo', 'adf_stat','adf_sign','coef_ang',
                                             'desvio','pct_fin','meia_vida', 'corr', 'fx_corr','per_coint','vol_beta_vertical','fx_vol_beta_vertical',
                                             'vol_beta_vertical1','fx_vol_beta_vertical1','adf99_periodos'])
    #arredondando o desvio
    cubo_of['desvio'] = cubo_of['desvio'].apply(lambda x: round(x,2))
    #criando o desvio com valor absoluto
    cubo_of['desvio_abs'] = cubo_of['desvio'].apply(lambda x: abs(x))
    #removendo elementos nao cointegrados adf_sign = '-'
    #cubo_of_index = cubo_of[ cubo_of['adf_sign'] == '-' ].index
    #cubo_of.drop(cubo_of_index , inplace=True)
    #removendo as linhas de close diario com hora = '19:00'
    cubo_of_index = cubo_of[ cubo_of['data'].apply(lambda x: x[-5:]) == '19-00' ].index
    cubo_of.drop(cubo_of_index , inplace=True)
    #definindo a data quarenta como a primeira data pra iniciar o loop
    dt_quarentena = date_unique[0]
    #selecionando linhas com padroes de DP e #perdiodos cointegrados e cointegradas    
    cubo_local =  cubo_of.loc[(cubo_of['desvio_abs'] >= dp) & (cubo_of['per_coint'] >= coint)&(cubo_of['adf_sign'] != '-')]

    
    for date in date_unique:
        
        var_trade = cubo_local [cubo_local['date_obj'] == date]
        
        #append do trade de cada iteracao, caso haja
        if len (var_trade) != 0:
            
            # pegando o index do maior valor
            adf_index = var_trade['adf_stat'].idxmin()

            #pegando a data do trade e criando o dt_obj da mesma , ESTE SERA O TRADE SELECIONADO
            trade_add = var_trade.loc[adf_index]
            data_trade = trade_add['data']
            d_obj_trade = trade_add['date_obj']
            #full_trade_select = full_trade_select.append(var_trade.iloc[0]) # remover dps do debug

            #verificando se a data do trade e maior que a data de quarentena de 1/4 de meia vida do trade anterior do par
            if (d_obj_trade >= dt_quarentena):
                
                trade_select = trade_select.append(trade_add)
                #criando o periodo de quarentana arredondado para cima
                dias_quarentena = int(math.ceil(trade_add['meia_vida'] / 4))
                #criando a date_obj da quarenta a ser reseitada em data util B3
                dt_quarentena = datetime.strptime(ajusta_B3Date(data_trade[:-6], dias_quarentena), '%Y-%m-%d-%H-%M')

            
    #print ('\n')
    
    #print ('TOTAL DE TRADES IDENTIFICADOS NO PERIODO  => {data}'.format(data = len(full_trade_select))) # remover dps do debug
    print ('TOTAL DE TRADES SELECIONADOS NO PERIODO   => {data}'.format(data = len(trade_select))) # remover dps do debug
    
    #display(full_trade_select)
    #display(trade_select)
    #full_trade_select.to_excel('Total_trades_Conga_BradesCAO.xlsx') # remover dps do debug
    
    return trade_select

In [9]:
def Simula_Trade (trade_select, cubototal, dp, debug = False):

    #removendo as linhas de close diario com hora = '19:00'
    cubototal_index = cubototal[ cubototal['data'].apply(lambda x: x[-5:]) == '19-00' ].index
    cubototal.drop(cubototal_index , inplace=True)

    #iteracao dentro dos trades selecionados
    if debug:
        trade_debug    = pd.DataFrame(columns=['index','data_trade','type','data_atual','cot_inicial_y','cot_inicial_x','ratio_trade','ratio_atual', 'ratio_gain','ratio_loss','coef_ang',
                                                 'cot_atual_x','cot_atual_y','fin_inicial', 'fin_atual', 'fin_acum','target_fin',
                                                 'status_ratio','status_fin','dt_saida_ratio','dt_saida_fin'])
        trade_debug.set_index('index')
        
        i = 0
        dict_debug = {}
           
    for index, trade in trade_select.iterrows():
        
        
        if debug:
            print ('****************************************************************')
            print ('******************INICIO DE ANALISE DO TRADE********************')
            print ('****************************************************************\n')
            i= i +1
            
        #criacao das datas limites para percorrer o trade
        dt_trade = datetime.strptime(trade['data'], '%Y-%m-%d-%H-%M')
        dt_meiavida = datetime.strptime(ajusta_B3Date(trade['data'][:-6], int(trade['meia_vida'])), '%Y-%m-%d-%H-%M')
        periodo_trade = trade['periodo'] #periodo do trade
        #selecionando o periodo do cubo entre as datas limites e o periodo correspondente ao trade selecionado
        cubo_slice = cubototal.loc[(cubototal['date_obj'] > dt_trade) & (cubototal['date_obj'] <= dt_meiavida) & (cubototal['periodo'] == periodo_trade)]
        #criando as variaveis do trade
        cot_x = trade['preco_x']
        cot_y = trade['preco_y']
        ratio_trade = trade['ratio']
        dp_residuo = trade['dp_residuo']*dp
        beta = trade['coef_ang']
        target_fin = dp_residuo
        
        #definindo as ratios conforme o trade tenha sido feito no desvio negativo ou positivo
        if trade_select.loc[index,'desvio'] > 0:
            desvio_positivo = True
            ratio_gain = (cot_y - dp_residuo)/cot_x
            ratio_loss = (cot_y + dp_residuo)/cot_x
            fin_trade = cot_y - (cot_x*beta)
        else:
            desvio_positivo = False
            ratio_gain = (cot_y + dp_residuo)/cot_x
            ratio_loss = (cot_y - dp_residuo)/cot_x
            fin_trade = (cot_x*beta) - cot_y
        
        #definindo variaveis controle que trade nao foi encontrado
        fin_status     = False
        ratio_status   = False
        fin_updated    = False
        ratio_updated  = False
        
        if debug:
            # Preenchimento do cubo
                dict_debug[i] = {
                    'index':i,
                    'data_trade' : dt_trade, 
                    'data_atual' : dt_trade,
                    'type' : 'Trade', 
                    'cot_inicial_y' : cot_y, 
                    'cot_inicial_x': cot_x, 
                    'cot_atual_y' : None, 
                    'cot_atual_x' : None, 
                    'ratio_trade' : ratio_trade, 
                    'ratio_atual' : None, 
                    'ratio_gain' : ratio_gain, 
                    'ratio_loss' : ratio_loss, 
                    'fin_inicial' : fin_trade, 
                    'target_fin': dp_residuo, 
                    'fin_atual': None, 
                    'fin_acum':None,
                    'status_ratio':None,
                    'status_fin':None, 
                    'dt_close_ratio': None ,
                    'fin_final_ratio' : None, 
                    'dt_close_fin':None, 
                    'fin_final_fin':None}

        #iterando o trade em questao dentro do periodo da meia-vida
        for var_index , trade_iterado in cubo_slice.iterrows():
            if debug:
                i = i + 1
            
            cot_x_atual = trade_iterado['preco_x']
            cot_y_atual = trade_iterado['preco_y']
            data_atual = datetime.strptime(trade_iterado['data'], '%Y-%m-%d-%H-%M')
        
            # Com desvio positivo, o trade inicial foi com venda de Y e compra de X e vice-versa
            if desvio_positivo:
                ratio_atual = (cot_y_atual)/cot_x_atual
                fin_atual =  (cot_x_atual*beta) - cot_y_atual
            else:
                ratio_atual = (cot_y_atual)/cot_x_atual
                fin_atual = cot_y_atual - (cot_x_atual*beta)
            
            #calculando o financeiro do trade no momento atual
            fin_acumulado =  fin_atual + fin_trade
            
            ######### inicio da analise com gain considerado o ratio projetado ########
            # Caso o desvio seja positivo no trade, Y sera vendido, entao o Gain sera quando a ratio for abaixo do Ratio Gain
            if (desvio_positivo) and (ratio_atual < ratio_gain) and (ratio_status == False):
                ratio_status = True
                result_ratio = 1
            elif (desvio_positivo) and (ratio_atual > ratio_loss) and (ratio_status == False):
                ratio_status = True
                result_ratio = 0
            elif (desvio_positivo == False) and (ratio_atual > ratio_gain) and (ratio_status == False):
                ratio_status = True
                result_ratio = 1
            elif (desvio_positivo == False) and (ratio_atual < ratio_loss) and (ratio_status == False):
                ratio_status = True
                result_ratio = 0
            
            #atualizando a Trade_select com o resultado fechado
            if ratio_status and ratio_updated == False:
                fin_final_ratio = fin_acumulado
                dt_close_ratio = data_atual
                days_elapsed_ratio = dt_close_ratio - dt_trade
                trade_select.loc[index,'result_trade_ratio'] = result_ratio
                trade_select.loc[index,'elapsed_days_ratio'] = days_elapsed_ratio.days 
                trade_select.loc[index,'result_ratio'] = fin_final_ratio
                trade_select.loc[index,'dt_close_ratio'] = dt_close_ratio
                ratio_updated = True
            
            ######### inicio da analise com gain considerado o financeiro calculado ########
            #verificando se o financeiro excedeu o target e se sera pela primeira vez (Fin_status == False)
            if (fin_acumulado > target_fin) and (fin_status == False):                
                result_fin = 1
                fin_status = True       
                
            elif (fin_acumulado < (target_fin*-1)) and (fin_status == False):    
                result_fin = 0
                fin_status = True
                
            if (fin_status) and (fin_updated == False):
                fin_updated = True
                fin_final_fin = fin_acumulado
                dt_close_fin = data_atual
                days_elapsed = dt_close_fin - dt_trade
                trade_select.loc[index,'result_trade_fin'] = result_fin
                trade_select.loc[index,'elapsed_days_fin'] = days_elapsed.days 
                trade_select.loc[index,'result_fin'] = fin_final_fin
                trade_select.loc[index,'dt_close_fin'] = dt_close_fin
                
            # INICIO DE DEGUB
            if debug:       

                if ratio_status == False:                   
                    dt_close_ratio  = None
                    fin_final_ratio = None
                
                if fin_status == False:                    
                    dt_close_fin = None
                    fin_final_fin = None

                # Preenchimento do debug
                dict_debug[i] = {
                    'index':i,
                    'data_trade' : dt_trade, 
                    'data_atual' : data_atual,
                    'type' : 'Iteracao', 
                    'cot_inicial_y' : cot_y, 
                    'cot_inicial_x': cot_x, 
                    'cot_atual_y' : cot_y_atual, 
                    'cot_atual_x' : cot_x_atual, 
                    'ratio_trade' : ratio_trade, 
                    'ratio_atual' : ratio_atual, 
                    'ratio_gain' : ratio_gain, 
                    'ratio_loss' : ratio_loss,
                    'fin_inicial' : fin_trade, 
                    'target_fin': dp_residuo, 
                    'fin_atual': fin_atual, 
                    'fin_acum':fin_acumulado,
                    'status_ratio':ratio_status,
                    'status_fin':fin_status, 
                    'dt_close_ratio': dt_close_ratio,
                    'fin_final_ratio' : fin_final_ratio, 
                    'dt_close_fin':dt_close_fin, 
                    'fin_final_fin':fin_final_fin}

            if fin_status and ratio_status:
                break
            
        #fim do for de procura do trade a meia vida
        if fin_status == False:
            trade_select.loc[index,'result_fin'] = fin_acumulado
            trade_select.loc[index,'dt_close_fin'] = data_atual
            days_elapsed = data_atual - dt_trade
            trade_select.loc[index,'elapsed_days_fin'] = days_elapsed.days
            if fin_acumulado >= 0:
                trade_select.loc[index,'result_trade_fin'] = 3
            else:
                trade_select.loc[index,'result_trade_fin'] = 4
        
        if ratio_status == False:
            trade_select.loc[index,'result_ratio'] = fin_acumulado
            trade_select.loc[index,'dt_close_ratio'] = data_atual
            days_elapsed_ratio = data_atual - dt_trade
            trade_select.loc[index,'elapsed_days_ratio'] = days_elapsed_ratio.days
            if fin_acumulado >= 0:
                trade_select.loc[index,'result_trade_ratio'] = 3
            else:
                trade_select.loc[index,'result_trade_ratio'] = 4  
    
    if debug:
        trade_debug = pd.DataFrame.from_dict(dict_debug, "index")
        trade_debug.to_excel('Trade_debug_Conga_BradesCAO.xlsx') # remover dps do debug
        #print (dict_debug)
        
    return trade_select

In [15]:
tickers = ['COGN3','BBDC4']

cubototal = geraCubodecubos(tickers, '2020-01-02','2020-06-30')

#cubototal.to_excel('cubo_Conga_BradesCAO.xlsx')

Tempo total em minutos = 7.338309462865194


In [10]:
cubototal =  pd.read_excel('cubo_Conga_BradesCAO.xlsx')

In [16]:
%time trades = Busca_trade (cubototal)

#trades.to_excel('Trades_adicionados_Conga_BradesCAO.xlsx')

TOTAL DE TRADES SELECIONADOS NO PERIODO   => 6
Wall time: 2.95 s


In [17]:
%time trades_atualizado = Simula_Trade(trades, cubototal, 1 , True) # Ultimo parametro (debug) como True para imprimir etapas

#display(trades_atualizado)
trades_atualizado.to_excel('Trades_closed_Conga_BradesCAO.xlsx')

****************************************************************
******************INICIO DE ANALISE DO TRADE********************
****************************************************************

****************************************************************
******************INICIO DE ANALISE DO TRADE********************
****************************************************************

****************************************************************
******************INICIO DE ANALISE DO TRADE********************
****************************************************************

****************************************************************
******************INICIO DE ANALISE DO TRADE********************
****************************************************************

****************************************************************
******************INICIO DE ANALISE DO TRADE********************
****************************************************************

********************

In [13]:
display(trades_atualizado)

,data,preco_y,preco_x,ratio,periodo,adf_stat,adf_sign,coef_ang,desvio,pct_fin,...,desvio_abs,dp_residuo,result_trade_ratio,elapsed_days_ratio,result_ratio,dt_close_ratio,result_trade_fin,elapsed_days_fin,result_fin,dt_close_fin
16308,2020-03-25-14-00,4.94,20.16,0.25,100,-4.46,99%,0.61,-2.00,40%,...,2.00,0.65,1.00,0.00,0.74,2020-03-26 11:45:00,1.00,0.00,0.74,2020-03-26 11:45:00
17406,2020-03-31-15-30,4.23,19.03,0.22,100,-4.43,99%,0.61,-2.03,36%,...,2.03,0.59,4.00,9.00,-0.42,2020-04-09 16:45:00,1.00,0.00,0.59,2020-04-01 12:30:00
18252,2020-04-06-10-15,3.93,18.97,0.21,100,-3.80,95%,0.62,-2.13,33%,...,2.13,0.61,1.00,9.00,0.30,2020-04-15 10:30:00,0.00,8.00,-0.61,2020-04-14 11:00:00
19018,2020-04-09-10-00,4.67,20.57,0.23,120,-3.78,95%,0.63,-2.16,36%,...,2.16,0.72,1.00,6.00,0.79,2020-04-15 11:45:00,1.00,6.00,0.72,2020-04-15 11:00:00
26759,2020-05-25-15-15,4.65,19.76,0.24,140,-4.12,95%,0.65,-2.02,36%,...,2.02,0.70,1.00,2.00,0.82,2020-05-27 16:45:00,1.00,1.00,0.71,2020-05-27 14:00:00
30729,2020-06-17-10-15,6.08,22.44,0.27,160,-3.47,90%,0.61,-2.12,44%,...,2.12,0.72,1.00,5.00,0.84,2020-06-22 11:30:00,1.00,5.00,0.77,2020-06-22 11:15:00


In [18]:
display(trades_atualizado)

,data,preco_y,preco_x,ratio,periodo,adf_stat,adf_sign,coef_ang,desvio,pct_fin,...,dp_residuo,per_coint99,result_trade_ratio,elapsed_days_ratio,result_ratio,dt_close_ratio,result_trade_fin,elapsed_days_fin,result_fin,dt_close_fin
16902,2020-03-25-14-00,4.94,20.16,0.25,100,-4.46,99%,0.61,-2.00,40%,...,0.65,8.00,1.00,0.00,0.74,2020-03-26 11:45:00,1.00,0.00,0.74,2020-03-26 11:45:00
17964,2020-03-31-15-30,4.23,19.03,0.22,100,-4.43,99%,0.61,-2.03,36%,...,0.59,8.00,4.00,9.00,-0.42,2020-04-09 16:45:00,1.00,0.00,0.59,2020-04-01 12:30:00
18774,2020-04-06-10-15,3.93,18.97,0.21,100,-3.80,95%,0.62,-2.13,33%,...,0.61,9.00,1.00,9.00,0.30,2020-04-15 10:30:00,0.00,8.00,-0.61,2020-04-14 11:00:00
19513,2020-04-09-10-00,4.67,20.57,0.23,120,-3.78,95%,0.63,-2.16,36%,...,0.72,9.00,1.00,6.00,0.79,2020-04-15 11:45:00,1.00,6.00,0.72,2020-04-15 11:00:00
26993,2020-05-25-15-15,4.65,19.76,0.24,140,-4.12,95%,0.65,-2.02,36%,...,0.70,9.00,1.00,2.00,0.82,2020-05-27 16:45:00,1.00,1.00,0.71,2020-05-27 14:00:00
30819,2020-06-17-10-15,6.08,22.44,0.27,160,-3.47,90%,0.61,-2.12,44%,...,0.72,8.00,1.00,5.00,0.84,2020-06-22 11:30:00,1.00,5.00,0.77,2020-06-22 11:15:00
